In [16]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import json

In [18]:
df=pd.read_csv('data_processed/subsetWyniki.csv')
df['htmlBody'] = df['htmlBody'].apply(lambda x: eval(x))

In [35]:
def one_hot_cpv(df):
    pattern = r'\d+-\d'
    matches = df['cpvCode'].apply(lambda x: re.findall(pattern, x))
    unique_matches = set([item for sublist in matches for item in sublist])
    for match in unique_matches:
        df[match] = df['cpvCode'].apply(lambda x: 1 if match in re.findall(pattern, x) else 0)

In [36]:
one_hot_cpv(df)
df.head()

,Unnamed: 0,orderType,bzpNumber,isTenderAmountBelowEU,orderObject,cpvCode,htmlBody,contractors,34114000-9,45111200-0,...,39222120-1,09135100-5,4.5.5.),6.3.),6.4.),8.2.),4.3.),4.3.1),6.2.),4.5.1.)
0,0,Services,2023/BZP 00000008,True,„Przewóz dzieci przedszkolnych i uczniów szkół...,60100000-9 (Usługi w zakresie transportu drogo...,"{'4.3.)': 431252.33, '4.3.1)': 399307.71, '4.5...",2,0,0,...,0,0,48159.36,74196.00,73440.00,73440.00,431252.33,399307.71,73440.00,0.0
1,1,Services,2023/BZP 00000010,True,Usługi restauracyjne na rzecz uczniów Szkoły P...,55523100-3 (Usługi w zakresie posiłków szkolny...,"{'6.2.)': 1210100.0, '6.3.)': 1383400.0, '6.4....",1,0,0,...,0,0,0.00,1383400.00,1103710.00,1103710.00,0.00,0.00,1210100.00,0.0
2,2,Services,2023/BZP 00000022,True,Zimowe utrzymanie obiektów drogowych: chodnikó...,90620000-9 (Usługi odśnieżania),"{'4.3.)': 438662.19, '8.2.)': 703677.89}",1,0,0,...,0,0,0.00,0.00,0.00,703677.89,438662.19,0.00,0.00,0.0
3,3,Works,2023/BZP 00000031,True,Remont 5 lokali mieszkalnych zasobu Gminy Miej...,"45453000-7 (Roboty remontowe i renowacyjne),45...","{'4.3.)': 7407407.41, '4.3.1)': 238562.68, '4....",5,0,0,...,0,0,38268.11,40594.45,37631.37,37631.37,7407407.41,238562.68,37631.37,0.0
4,4,Services,2023/BZP 00000037,True,Świadczenie usług utrzymania czystości.,"90919200-4 (Usługi sprzątania biur),90919100-3...","{'4.3.)': 229952.0, '4.5.5.)': 109584.0, '6.2....",1,0,0,...,0,0,109584.00,172188.00,108999.72,108999.72,229952.00,0.00,108999.72,0.0


In [34]:
unique_keys = set(key for d in df["htmlBody"] for key in d.keys())
expanded_df = pd.DataFrame(
    [{key: d.get(key, 0) for key in unique_keys} for d in df["htmlBody"]]
)
df = pd.concat([df, expanded_df], axis=1)

In [37]:
df.drop(columns=["cpvCode", "htmlBody"], inplace=True)

In [38]:
df.head()

,Unnamed: 0,orderType,bzpNumber,isTenderAmountBelowEU,orderObject,contractors,34114000-9,45111200-0,79822000-2,15131400-9,...,39222120-1,09135100-5,4.5.5.),6.3.),6.4.),8.2.),4.3.),4.3.1),6.2.),4.5.1.)
0,0,Services,2023/BZP 00000008,True,„Przewóz dzieci przedszkolnych i uczniów szkół...,2,0,0,0,0,...,0,0,48159.36,74196.00,73440.00,73440.00,431252.33,399307.71,73440.00,0.0
1,1,Services,2023/BZP 00000010,True,Usługi restauracyjne na rzecz uczniów Szkoły P...,1,0,0,0,0,...,0,0,0.00,1383400.00,1103710.00,1103710.00,0.00,0.00,1210100.00,0.0
2,2,Services,2023/BZP 00000022,True,Zimowe utrzymanie obiektów drogowych: chodnikó...,1,0,0,0,0,...,0,0,0.00,0.00,0.00,703677.89,438662.19,0.00,0.00,0.0
3,3,Works,2023/BZP 00000031,True,Remont 5 lokali mieszkalnych zasobu Gminy Miej...,5,0,0,0,0,...,0,0,38268.11,40594.45,37631.37,37631.37,7407407.41,238562.68,37631.37,0.0
4,4,Services,2023/BZP 00000037,True,Świadczenie usług utrzymania czystości.,1,0,0,0,0,...,0,0,109584.00,172188.00,108999.72,108999.72,229952.00,0.00,108999.72,0.0


In [39]:
df.to_csv('data_processed/subsetWyniki10_oneHot.csv')